# Matrix Decomposition

## Contents:
 * [Gauss Elimination](#Gauss-Elimination)
 * [Gaussian Decomposition](#Gaussian-Decomposition)
     * [$LU$](#$A-=-LU$-decomposition)
     * [$LDU$](#$A-=-LDU$-decomposition)
     * [$LDL^t$](#$A-=-LDL^t$-decomposition)
     * [$RR^t$](#$A-=-RR^t$-decomposition)
 * [Orthogonal Decomposition](#Orthogonal-Decomposition)
     * [$QR$ - Givens](#$A-=-QR$-decomposition-of-Householder)
     * [$QR$ - Householder](#$A-=-QR$-decomposition-of-Householder)
 * [Diagonal Decomposition](#Diagonal-Decomposition)
     * [$U \Sigma V^t$](#$A-=-U\Sigma-V^t$-decomposition)

#### Author:
* Sergio García Prado - [garciparedes.me](https://garciparedes.me)

In [1]:
from typing import Tuple
import enforce

In [2]:
import numpy as np

## Gauss Elimination

In [3]:
@enforce.runtime_validation
def row_reduce(A: np.ndarray, upper_reduce: bool = True) -> np.ndarray:
    A = A.astype(float)
    for i in range(A.shape[0]):
        if (A[i, i] != 0):
            for j in reversed(range(i + 1, A.shape[0])):
                A[j,:] -= A[j, i] / A[i, i] * A[i,:]    
    if upper_reduce is True:
        i = A.shape[0] - 1
        if A[i, i] != 0:
            A[i, :] = A[i, :] / A[i, i]
        for i in reversed(range(A.shape[0] - 1)):
            if (A[i, i] != 0):
                A[i, :] = A[i, :] / A[i, i]
                for j in range(i+1, A.shape[0]):
                    if(A[j, j] != 0):
                        A[i, :] -= A[i,j] / A[j, j] * A[j,:]
    return A

In [4]:
@enforce.runtime_validation
def gauss(A: np.ndarray, b: np.ndarray, generate_underdetermined:bool = True) -> np.ndarray:
    R = row_reduce(np.column_stack((A, b)))
    
    if not generate_underdetermined:
        return R[:, -1]
    else:
        # TODO(@garciparedes): Improve this code.
        i = A.shape[1] - 1
        free = 0
        while R[i, i] == 0:
            i -= 1
            free += 1
        
        if free == 0:
            c = R[:,-1]
        else:
            c = np.empty([A.shape[0], free])
            for i in range(free):
                R_copy = np.copy(R)
                for j in range(free):
                    R_copy[A.shape[1] - j - 1, A.shape[1] - j - 1] = 1
                    R_copy[A.shape[1] - j -1 , - 1] = i + j + 1
                #print(R_copy)
                c[:, i] = row_reduce(R_copy)[:, -1]   
        return c

In [5]:
A = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

b = np.ones((3,1))
x = gauss(A, b)

print("A =", np.round(A, decimals=2),  '\n', sep='\n')
print("b =", np.round(b, decimals=2),  '\n', sep='\n')
print("x =", np.round(x, decimals=2),  '\n', sep='\n')
print("A @ x =", np.round(A @ x, decimals=2), '\n', sep='\n')
print("A @ x == b ?", np.allclose(A @ x, b), '\n', sep='\n')

A =
[[1 2 3]
 [4 5 6]
 [7 8 9]]


b =
[[1.]
 [1.]
 [1.]]


x =
[[ 0.]
 [-1.]
 [ 1.]]


A @ x =
[[1.]
 [1.]
 [1.]]


A @ x == b ?
True




## Gaussian Decomposition

#### $A = LU$ decomposition

In [6]:
@enforce.runtime_validation
def lu(A: np.ndarray, method: str = "doolittle") -> Tuple[np.ndarray, np.ndarray]:
    A = A.astype(np.float64)
    
    if method is "doolittle":
        pass
    elif method is "crout":
        A = A.T
    else:
        raise ValueError("LU decomposition method is not properly especified: " + str(method))
    
    loop_i = range(A.shape[0])
    loop_j = lambda i: reversed(range(i +1, A.shape[0]))
        
    for i in loop_i:
        if (A[i, i] != 0):
            for j in loop_j(i):
                A[j, (i+1):] = A[j, (i+1):] - A[j, i] / A[i, i] * A[i, (i+1):]
                A[j, i] = A[j, i] / A[i, i]
    
    if method is "doolittle":
        return (np.tril(A, -1) + np.eye(A.shape[0]), np.triu(A))
    elif method is "crout":
        A = A.T
        return (np.tril(A), np.triu(A, 1) + np.eye(A.shape[0]))
    

In [7]:
A = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

print("A =", np.round(A, decimals=2),  '\n', sep='\n')


(L, U) = lu(A)

print("L =", np.round(L, decimals=2),  '\n', sep='\n')
print("U =", np.round(U, decimals=2),  '\n', sep='\n')
print("L @ U == A ?", np.allclose(L @ U, A), '\n', sep='\n')


(L, U) = lu(A, method="crout")

print("L =", np.round(L, decimals=2),  '\n', sep='\n')
print("U =", np.round(U, decimals=2),  '\n', sep='\n')
print("L @ U == A ?", np.allclose(L @ U, A), '\n', sep='\n')



A =
[[1 2 3]
 [4 5 6]
 [7 8 9]]


L =
[[1. 0. 0.]
 [4. 1. 0.]
 [7. 2. 1.]]


U =
[[ 1.  2.  3.]
 [ 0. -3. -6.]
 [ 0.  0.  0.]]


L @ U == A ?
True


L =
[[ 1.  0.  0.]
 [ 4. -3.  0.]
 [ 7. -6.  0.]]


U =
[[1. 2. 3.]
 [0. 1. 2.]
 [0. 0. 1.]]


L @ U == A ?
True




In [8]:
@enforce.runtime_validation
def gauss_lu(LU: Tuple[np.ndarray, np.ndarray], b: np.ndarray) -> np.ndarray:
    # TODO(@garciparedes): Fix bug that cannot gives appropiate result
    raise NotImplementedError("It's necessary to fix some bugs before give any result.")
    for i in range(LU[0].shape[0]):
        b[i] -= np.sum(b[:i] * LU[0][i,:i])
    for i in reversed(range(LU[1].shape[0])):
        if LU[1][i][i] != 0:
            b[i] = b[i] / LU[1][i][i] - np.sum(b[(i + 1):] * LU[1][i, (i + 1):])
    return b

In [9]:
A = np.array([[2,  0,  1,  2,],
              [1,  1,  0,  2,],
              [2, -1,  3,  1,],
              [3, -1,  4,  3,]])

row_reduce(np.column_stack((A, np.ones(4))))

# print(A @ gauss_lu(lu(A), np.ones(4)), sep = '\n')
print(A @ gauss(A, np.ones(4)))

[1. 1. 1. 1.]


In [10]:
L = np.array([[1, 0, 0, 0], 
              [1, 1, 0, 0],
              [2,-1, 1, 0],
              [-1,1, 0, 1]])

U = np.array([[1, 1, -1, 1], 
              [0, 1, -3, -3],
              [0, 0, 0, 0],
              [0, 0, 0, 0]])

P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
A = P @ L @ U
print(P @ L @ U)

[[ 1  1 -1  1]
 [ 1  2 -4 -2]
 [ 2  1  1  5]
 [-1  0 -2 -4]]


In [11]:
L = np.array([[  1,    0,   0,  0], 
              [ -1,    1,   0,  0],
              [3/2, 5/14,   1,  0],
              [  3,    0,   0,  1]])

U = np.array([[ 2,     3,   1,   2], 
              [ 0,     7,   0,   7],
              [ 0,     0,   0,-9/2],
              [ 0,     0,   0,   1]])

P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

print(P @ L @ U)

[[ 2.   3.   1.   2. ]
 [-2.   4.  -1.   5. ]
 [ 3.   7.   1.5  1. ]
 [ 6.   9.   3.   7. ]]


In [12]:
L = np.array([[  1,   0,   0], 
              [3/4,   1,   0],
              [-1/2,  0,   1]])

U = np.array([[ 4,    2,   6], 
              [ 0, -3/2, 5/2],
              [ 0,     0, 0]])

P = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

print(P @ L @ U)

[[ 4.  2.  6.]
 [ 3.  0.  7.]
 [-2. -1. -3.]]


In [13]:
L = np.array([[  1,    0,   0,  0], 
              [1/2,   1,   0,  0],
              [  1,   -1,   1,  0],
              [3/2,   -1, 4/3,  1]])

U = np.array([[ 2,  0,   1,   2], 
              [ 0,  1,-1/2,   1],
              [ 0,  0, 3/2,   0],
              [ 0,  0,   0,   1]])

P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

print(P @ L @ U)

[[ 2.  0.  1.  2.]
 [ 1.  1.  0.  2.]
 [ 2. -1.  3.  1.]
 [ 3. -1.  4.  3.]]


In [14]:
L = np.array([[1, 0, 0, 0], 
              [-1, 1, 0, 0],
              [3, 1/2, 1, 0],
              [2, 0, 0, 1]])

U = np.array([[1, 2, 0, -2], 
              [0, 2, 4, 5],
              [0, 0, 0, 1/2],
              [0, 0, 0, 1]])

P = np.array([[1, 0, 0, 0],
              [0, 0, 0, 1],
              [0, 0, 1, 0],
              [0, 1, 0, 0]])

print(P @ L @ U)

[[ 1.  2.  0. -2.]
 [ 2.  4.  0. -3.]
 [ 3.  7.  2. -3.]
 [-1.  0.  4.  7.]]


## Gaussian Decomposition

#### $A = LDU$ decomposition

In [15]:
@enforce.runtime_validation
def ldu(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:    
    (L, U1) = lu(A)
    D = np.diag(np.diag(U1))
    U = np.linalg.pinv(D) @ U1
    return (L, D, U)
    

In [16]:
A = np.array([[ 4, -2,  1],
              [20, -7, 12],
              [-8, 13, 17]])

print("L =", np.round(U, decimals=2),  '\n', sep='\n')


(L, D, U) = ldu(A)

print("L =", np.round(L, decimals=2),  '\n', sep='\n')
print("D =", np.round(D, decimals=2),  '\n', sep='\n')
print("U =", np.round(U, decimals=2),  '\n', sep='\n')
print("L @ D @ U == A ?", np.allclose(L @ D @ U, A), '\n', sep='\n')


L =
[[ 1.   2.   0.  -2. ]
 [ 0.   2.   4.   5. ]
 [ 0.   0.   0.   0.5]
 [ 0.   0.   0.   1. ]]


L =
[[ 1.  0.  0.]
 [ 5.  1.  0.]
 [-2.  3.  1.]]


D =
[[ 4.  0.  0.]
 [ 0.  3.  0.]
 [ 0.  0. -2.]]


U =
[[ 1.   -0.5   0.25]
 [ 0.    1.    2.33]
 [ 0.    0.    1.  ]]


L @ D @ U == A ?
True




In [17]:
L = np.array([[  1,   0,   0], 
              [  5,   1,   0],
              [ -2,   3,   1]])

U = np.array([[  4,  -2,   1], 
               [ 0,   3,   7],
               [ 0,   0,  -2]])

D = np.array([[ 4, 0, 0], 
              [0, 3, 0],
              [0, 0,-2]])

P = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

print(P @ L @ U)

[[ 4 -2  1]
 [20 -7 12]
 [-8 13 17]]


In [18]:
L = np.array([[1, 0, 0, 0], 
              [4, 1, 0, 0],
              [6, 8, 1, 0],
              [5, -5, 0, 1]])

U = np.array([[1, 2, -2, 1],
              [0, -3, 1, 2],
              [0, 0, -2, 0],
              [0, 0, 0, 8]])

D = np.array([[1, 0, 0, 0],
              [0, -3, 0, 0],
              [0, 0, -2, 0],
              [0, 0, 0, 8]])

P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

print(P @ L @ U)

[[  1   2  -2   1]
 [  4   5  -7   6]
 [  6 -12  -6  22]
 [  5  25 -15   3]]


In [19]:
L = np.array([[  1,   0,   0], 
              [  5,   1,   0],
              [ -2,   3,   1]])

D = np.array([[ 4, 0, 0], 
              [0, 3, 0],
              [0, 0,-2]])

U = np.array([[ 1,-1/2,1/4], 
               [ 0,   1,7/3],
               [ 0,   0,  1]])


P = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

print(P @ L @ D @ U)

[[ 4. -2.  1.]
 [20. -7. 12.]
 [-8. 13. 17.]]


In [20]:
L = np.array([[1, 0, 0, 0], 
              [4, 1, 0, 0],
              [6, 8, 1, 0],
              [5, -5, 0, 1]])

D = np.array([[1, 0, 0, 0],
              [0, -3, 0, 0],
              [0, 0, -2, 0],
              [0, 0, 0, 8]])

U = np.array([[1, 2, -2, 1],
               [0, 1, -1/3, -2/3],
               [0, 0, 1, 0],
               [0, 0, 0, 1]])


P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

print(P @ L @ D @ U)

[[  1.   2.  -2.   1.]
 [  4.   5.  -7.   6.]
 [  6. -12.  -6.  22.]
 [  5.  25. -15.   3.]]


In [21]:
L = np.array([[1, 0, 0, 0, 0], 
              [0, 1, 0, 0, 0],
              [4, 2, 1, 0, 0],
              [5, 3, -1, 1, 0],
              [0, 0, 0, 0, 1]])

U = np.array([[1, 2, -2, 1, 1],
              [0, 1, -1/3, -2/3, -2/3],
              [0, 0, 1, 0, -1/2],
              [0, 0, 0, 1, 1/2],
              [0, 0, 0, 0, 1]])

D = np.array([[1, 0, 0, 0, 0],
              [0, -3, 0, 0, 0],
              [0, 0, -2, 0, 0],
              [0, 0, 0, 2, 0],
              [0, 0, 0, 0, 2]])

P = np.array([[1, 0, 0, 0, 0],
              [0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1]])

print(P @ L @ D @ U)

[[ 1.  2. -2.  1.  1.]
 [ 0. -3.  1.  2.  2.]
 [ 4.  2. -8.  8.  9.]
 [ 5.  1. -5. 13. 11.]
 [ 0.  0.  0.  0.  2.]]


#### $A = LDL^t$ decomposition

In [22]:
@enforce.runtime_validation
def ldl(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    L = np.zeros(A.shape)
    D = np.zeros(A.shape)
    
    raise NotImplementedError("TODO IMPLEMENT YET LDL")

In [23]:
L = np.array([[1, 0, 0], 
              [-1/2, 1, 0],
              [-2, -9/8, 1]])

D = np.array([[4, 0, 0], 
              [0, -8, 0],
              [0, 0, 89/8]])

P = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

print(P @ L @ D @ np.transpose(L))

[[ 4. -2. -8.]
 [-2. -7. 13.]
 [-8. 13. 17.]]


In [24]:
L = np.array([[1, 0, 0, 0], 
              [4, 1, 0, 0],
              [-2, 4/11, 1, 0],
              [1, -21/11, 227/94, 1]])

D = np.array([[1, 0, 0, 0], 
              [0, -11, 0, 0],
              [0, 0, -94/11, 0],
              [0, 0, 0, 8077/94]])

P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

print(P @ L @ D @ np.transpose(L))

[[  1.   4.  -2.   1.]
 [  4.   5. -12.  25.]
 [ -2. -12.  -6. -15.]
 [  1.  25. -15.  -3.]]


In [25]:
L = np.array([[1, 0, 0, 0], 
              [4, 1, 0, 0],
              [-2, 4/11, 1, 0],
              [1, -21/11, 227/94, 1]])

D = np.array([[1, 0, 0, 0], 
              [0, -11, 0, 0],
              [0, 0, -94/11, 0],
              [0, 0, 0, 8077/94]])

P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

print(P @ L @ D @ np.transpose(L))

[[  1.   4.  -2.   1.]
 [  4.   5. -12.  25.]
 [ -2. -12.  -6. -15.]
 [  1.  25. -15.  -3.]]


In [26]:
L = np.array([[1, 0, 0, 0, 0], 
              [2, 1, 0, 0, 0],
              [4, 6/7, 1, 0, 0],
              [5, 8/7, 127/132, 1, 0],
              [1, 0, -35/132, -1163/739, 1]])

D = np.array([[1, 0, 0, 0, 0], 
              [0, -7, 0, 0, 0],
              [0, 0, -132/7, 0, 0],
              [0, 0, 0, 739/132, 0],
              [0, 0, 0, 0,  -8528/739]])

P = np.array([[1, 0, 0, 0, 0],
              [0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1]])

print(P @ L @ D @ np.transpose(L))

[[ 1.  2.  4.  5.  1.]
 [ 2. -3.  2.  2.  2.]
 [ 4.  2. -8. -5.  9.]
 [ 5.  2. -5.  4.  1.]
 [ 1.  2.  9.  1.  2.]]


## $A = RR^t$ decomposition

In [27]:
@enforce.runtime_validation
def rr(A: np.ndarray) -> Tuple[np.ndarray]:
    R = np.zeros(A.shape)
    
    raise NotImplementedError("TODO IMPLEMENT YET: RR")

In [28]:
R = np.array([[2, 0, 0],
              [1, 3, 0],
              [-2, 2, 1]])

P = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

print(P @ R @ np.transpose(R))

[[ 4  2 -4]
 [ 2 10  4]
 [-4  4  9]]


In [29]:
R = np.array([[2, 0, 0],
              [1, np.sqrt(8), 0],
              [0, np.sqrt(8) / 2, np.sqrt(3)]])

P = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

print(P @ R @ np.transpose(R))

[[4. 2. 0.]
 [2. 9. 4.]
 [0. 4. 5.]]


## Orthogonal Decomposition

#### $A = QR$ decomposition of Gram Schmidt

In [30]:
def inner_product(v1, v2):
    return np.sum(v1 * v2)

In [31]:
def norm_2(v):
    return np.sqrt(inner_product(v, v))

In [32]:
def normalize(v):
    return v / norm_2(v)

In [33]:
def gram_schmidt(V):
    Q = np.zeros(V.shape)
    for i in range(Q.shape[1]):
        Q[:, i] = V[:, i] 
        for j in range(i):
            Q[:, i] -= inner_product(Q[:, j], Q[:, i]) * Q[:, j]
        Q[:, i] = normalize(Q[:, i]) 
    return Q

#### $A = QR$ decomposition of Givens 

In [34]:
@enforce.runtime_validation
def rotator(A: np.ndarray, i: int, j: int, k: int) -> np.ndarray:
    G = np.eye(A.shape[0])
    r = np.sqrt(A[i, k] ** 2 + A[j, k] ** 2)
    if A[j, k] != 0:
        G[i, i] = A[i, k] / r
        G[i, j] = A[j, k] / r
        G[j, i] = - A[j, k] / r
        G[j, j] = A[i, k] / r
    return G

In [35]:
@enforce.runtime_validation
def qr_givens(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    Q = np.eye(A.shape[0])
    for j in range(A.shape[1]):
        G = np.eye(A.shape[0])
        for i in reversed(range(j, A.shape[1])):
            G = rotator(G @ A, i, i + 1, j) @ G
        Q = Q @ np.transpose(G)
        A = G @ A
    return (Q, A)

In [36]:
A = np.array([[1,-1,1],
              [1,1,1],
              [1,1,-1],
             [1,1,1]])

b = np.array([1, 1, -1, 0])
[Q, R] = qr_givens(A)
c = (np.transpose(Q) @ b).flatten()
x = np.linalg.solve(R[0:R.shape[1], :], c[0:R.shape[1]])

print("A =", np.round(A, decimals=2),  '\n', sep='\n')
print("Q =", np.round(Q, decimals=2),  '\n', sep='\n')
print("R =", np.round(R, decimals=2),  '\n', sep='\n')
print("Q @ R =", np.round(Q @ R, decimals=2), '\n', sep='\n')
print("b =", np.round(b, decimals=2), '\n', sep='\n')
print("c =", np.round(c, decimals=2), '\n', sep='\n')
print("x =", np.round(x, decimals=2), '\n', sep='\n')

A =
[[ 1 -1  1]
 [ 1  1  1]
 [ 1  1 -1]
 [ 1  1  1]]


Q =
[[ 0.5  -0.87  0.    0.  ]
 [ 0.5   0.29  0.41  0.71]
 [ 0.5   0.29 -0.82  0.  ]
 [ 0.5   0.29  0.41 -0.71]]


R =
[[ 2.    1.    1.  ]
 [ 0.    1.73 -0.58]
 [ 0.    0.    1.63]
 [ 0.    0.    0.  ]]


Q @ R =
[[ 1. -1.  1.]
 [ 1.  1.  1.]
 [ 1.  1. -1.]
 [ 1.  1.  1.]]


b =
[ 1  1 -1  0]


c =
[ 0.5  -0.87  1.22  0.71]


x =
[ 0.   -0.25  0.75]




#### $A = QR$ decomposition of Householder

In [37]:
@enforce.runtime_validation
def reflector(A: np.ndarray, i: int) -> np.ndarray:
    x = np.take(A, [i], 1)
    y =  np.concatenate([x[:i], [[np.sqrt(np.sum(np.power(x[i:],2)))]],
                        np.zeros([A.shape[0] - i - 1, 1])])
    u = (x - y)
    return np.eye(A.shape[0]) - (2 / np.squeeze(u.T @ u)) * (u @ u.T)

In [38]:
@enforce.runtime_validation
def qr_householder(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    Q = np.eye(A.shape[0])
    for i in range(A.shape[1]):
        R = reflector(A, i)
        Q = Q @ R
        A = R @ A
    return (Q, A)

In [39]:
A = np.array([[1,-1,1],
                [1,1,1],
                [1,1,-1],
                [1,1,1]])

b = np.array([1, 1, -1, 0])

[Q, R] = qr_householder(A)
c = (np.transpose(Q) @ b).flatten()
x = np.linalg.solve(R[0:R.shape[1], :], c[0:R.shape[1]])

print("A =", np.round(A, decimals=2),  '\n', sep='\n')
print("Q =", np.round(Q, decimals=2),  '\n', sep='\n')
print("R =", np.round(R, decimals=2),  '\n', sep='\n')
print("Q @ R =", np.round(Q @ R, decimals=2), '\n', sep='\n')
print("b =", np.round(b, decimals=2), '\n', sep='\n')
print("c =", np.round(c, decimals=2), '\n', sep='\n')
print("x =", np.round(x, decimals=2), '\n', sep='\n')

A =
[[ 1 -1  1]
 [ 1  1  1]
 [ 1  1 -1]
 [ 1  1  1]]


Q =
[[ 0.5  -0.87  0.   -0.  ]
 [ 0.5   0.29  0.41 -0.71]
 [ 0.5   0.29 -0.82  0.  ]
 [ 0.5   0.29  0.41  0.71]]


R =
[[ 2.    1.    1.  ]
 [ 0.    1.73 -0.58]
 [ 0.   -0.    1.63]
 [ 0.    0.    0.  ]]


Q @ R =
[[ 1. -1.  1.]
 [ 1.  1.  1.]
 [ 1.  1. -1.]
 [ 1.  1.  1.]]


b =
[ 1  1 -1  0]


c =
[ 0.5  -0.87  1.22 -0.71]


x =
[ 0.   -0.25  0.75]




## Diagonal Decomposition

#### $A = U\Sigma V^t$ decomposition

In [40]:
def eig_vals(A):
    # TODO(@garciparedes): Replace rounding with appropriate approach
    return np.round(np.linalg.eigvals(A), decimals = 4)

In [41]:
@enforce.runtime_validation
def usv(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    A1 = A.T @ A
    A2 = A @ A.T
    
    lambdas = np.sort(eig_vals(A1))

    S = np.diag(np.sqrt(lambdas))
    
    V = np.empty((A1.shape[1], 0), np.float64)
    for l in np.sort(np.unique(lambdas)):
        V = np.append(V, gauss(A1 - l * np.eye(A1.shape[0]), np.zeros(A1.shape[0])), axis=1)
    V = gram_schmidt(V)

    U = np.empty((A2.shape[1], 0), float)
    for (i, l) in enumerate(lambdas):
        if l == 0:
            U = np.append(U, gram_schmidt(gauss(A2, np.zeros(A2.shape[0]))), axis=1)
        else:
            U = np.append(U, 1 / np.sqrt(l) * A @ V[:, i, None], axis=1)
            
    return (U, S, V)

In [42]:
def pseudo_inverse(U, S, V):
    # TODO(@garciparedes): Apply reciprocal in S only over non-zero elements.
    return V @ np.linalg.pinv(S) @ U.T

In [43]:
A = np.array([[1, -1, 2],
              [1, 2, -1],
              [2, 1, 1]])

[U, S, V]  = usv(A)

b = np.array([[1], 
              [2], 
              [3]])

A_plus = pseudo_inverse(U, S, V)

x = A_plus @ b

print("A =", np.round(A, decimals=2),  '\n', sep='\n')
print("U =", np.round(U, decimals=2),  '\n', sep='\n')
print("S =", np.round(S, decimals=2),  '\n', sep='\n')
print("V =", np.round(V, decimals=2),  '\n', sep='\n')
print("U @ S @ V.T =", np.round(U @ S @ V.T, decimals=2),  '\n', sep='\n')
print("U @ S @ V.T == A ?", np.allclose(U @ S @ V.T, A), '\n', sep='\n')
print("A_plus =", np.round(A_plus, decimals=2),  '\n', sep='\n')
print("b =", np.round(b, decimals=2),  '\n', sep='\n')
print("x =", np.round(x, decimals=2),  '\n', sep='\n')
print("A @ x == b ?", np.allclose(A @ x, b), '\n', sep='\n')

A =
[[ 1 -1  2]
 [ 1  2 -1]
 [ 2  1  1]]


U =
[[-0.58  0.27  0.77]
 [-0.58  0.53 -0.62]
 [ 0.58  0.8   0.15]]


S =
[[-0.  0.  0.]
 [ 0.  3.  0.]
 [ 0.  0.  3.]]


V =
[[-0.58  0.8   0.15]
 [ 0.58  0.53 -0.62]
 [ 0.58  0.27  0.77]]


U @ S @ V.T =
[[ 1. -1.  2.]
 [ 1.  2. -1.]
 [ 2.  1.  1.]]


U @ S @ V.T == A ?
True


A_plus =
[[ 0.11  0.11  0.22]
 [-0.11  0.22  0.11]
 [ 0.22 -0.11  0.11]]


b =
[[1]
 [2]
 [3]]


x =
[[1.  ]
 [0.67]
 [0.33]]


A @ x == b ?
True


